# PCB Defect Detector Training

**Before running:**
1. Add dataset: `akhatova/pcb-defects`
2. Enable GPU: Settings → Accelerator → GPU

In [ ]:
# Install dependencies (if needed)
!pip install -q tensorflow scikit-learn matplotlib seaborn

In [ ]:
# Clone repository
import os
if not os.path.exists('pcb-defect-detector'):
    !git clone https://github.com/alainpaluku/pcb-defect-detector.git
    
import sys
sys.path.insert(0, 'pcb-defect-detector')

In [ ]:
# Import modules
from pathlib import Path
from config import PipelineConfig, DataConfig, ModelConfig, TrainingConfig
from data_manager import KaggleDataManager
from data_pipeline import DataPipeline
from model_builder import PCBModelBuilder
from trainer import Trainer
from evaluator import Evaluator

In [ ]:
# Configuration
config = PipelineConfig(
    data=DataConfig(
        data_dir=Path('/kaggle/input/pcb-defects'),
        batch_size=32,
        image_size=(224, 224)
    ),
    model=ModelConfig(
        base_model='MobileNetV2',
        dropout_rate=0.5
    ),
    training=TrainingConfig(
        epochs=25,
        fine_tune_epochs=15,
        checkpoint_dir=Path('/kaggle/working/checkpoints')
    ),
    results_dir=Path('/kaggle/working/results')
)

In [ ]:
# Load dataset
data_manager = KaggleDataManager(config.data)
data_manager.download_dataset()
class_images = data_manager.parse_directory_structure()
class_names = data_manager.get_class_names()
print(f"Classes: {class_names}")

In [ ]:
# Prepare data
data_pipeline = DataPipeline(config.data, class_names)
data_pipeline.prepare_data(class_images)
print(f"Splits: {data_pipeline.get_num_samples()}")

In [ ]:
# Build model
model_builder = PCBModelBuilder(config.model, config.data, len(class_names))
model = model_builder.build()
model.summary()

In [ ]:
# Train (frozen base)
trainer = Trainer(config.training, model)
trainer.compile()
trainer.train(
    data_pipeline.get_train_dataset(),
    data_pipeline.get_val_dataset(),
    data_pipeline.get_class_weights()
)

In [ ]:
# Fine-tune
model_builder.unfreeze_layers(30)
trainer.fine_tune(
    data_pipeline.get_train_dataset(),
    data_pipeline.get_val_dataset(),
    data_pipeline.get_class_weights()
)

In [ ]:
# Evaluate
evaluator = Evaluator(model, class_names, config.results_dir)
results = evaluator.generate_full_report(
    data_pipeline.get_test_dataset(),
    data_pipeline.get_test_labels(),
    trainer.get_combined_history(),
    data_pipeline.get_test_paths()
)

print(f"\nTest Accuracy: {results['test_accuracy']:.4f}")
print(f"F1 Macro: {results['f1_macro']:.4f}")

In [ ]:
# Display results
from IPython.display import Image, display

display(Image(str(config.results_dir / 'training_curves.png')))
display(Image(str(config.results_dir / 'confusion_matrix_normalized.png')))
display(Image(str(config.results_dir / 'misclassified.png')))